In [1]:
import sqlite3
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../module1-introduction-to-sql')

from rpg_queries import RPG

In [2]:
db = 'rpg_db.sqlite3'
characters_table = 'charactercreator_character'
weapons_table = 'armory_weapon'
items_table = 'armory_item'
inventory_table = 'charactercreator_character_inventory'
subclass_tables = ['charactercreator_cleric', 'charactercreator_fighter',
                   'charactercreator_mage', 'charactercreator_thief']

rpg = RPG(db)

In [109]:
character_column_names, characters = rpg.fetchall(f"SELECT * FROM {characters_table}", True)
characters = {
            character[0]: {column_name: value for column_name, value in
                           zip(character_column_names, character)} for
            character in characters}

In [110]:
characters[1]

{'character_id': 1,
 'name': 'Aliquid iste optio reiciendi',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1}

In [111]:
subclass_data = {}

for table in subclass_tables:
    subclass = table.split("_")[1]
    
    columns, rows = rpg.fetchall(f"SELECT * FROM {table}", True)
    
    data = {row[0]: {column_name: row for column_name, row in zip(columns[1:], row[1:])} for row in rows}
    
    subclass_data[subclass] = data

In [112]:
for subclass, rows in subclass_data.items():
    for id in rows.keys():
        characters[id][subclass] = rows[id]

In [113]:
characters[1]

{'character_id': 1,
 'name': 'Aliquid iste optio reiciendi',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'fighter': {'using_shield': 0, 'rage': 100}}

In [114]:
columns, rows = rpg.fetchall(f"SELECT character_id, INV.item_id, name, value, weight FROM {items_table} as IT JOIN {inventory_table} as INV ON INV.item_id = IT.item_id", True)

In [115]:
columns

['character_id', 'item_id', 'name', 'value', 'weight']

In [116]:
len(rows)

898

In [117]:
inventory = {}

for item in rows:
    item_dict = {column: value for column, value in zip(columns[1:], item[1:])}
    if item[0] in inventory:
        inventory[item[0]].append(item_dict)
    else:
        inventory[item[0]] = [item_dict]

In [118]:
for character_id, items in inventory.items():
    characters[character_id]['inventory'] = items

In [119]:
characters[3]

{'character_id': 3,
 'name': 'Minus c',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'fighter': {'using_shield': 0, 'rage': 100},
 'inventory': [{'item_id': 8, 'name': 'Saepe ea vo', 'value': 0, 'weight': 0},
  {'item_id': 43, 'name': 'Tempore', 'value': 0, 'weight': 0}]}

In [120]:
columns, rows = rpg.fetchall(f"SELECT * FROM {weapons_table}", True)

weapons = {row[0]: {column:value for column, value in zip(columns[1:], row[1:])} for row in rows}
weapons

{138: {'power': 0},
 139: {'power': 0},
 140: {'power': 0},
 141: {'power': 0},
 142: {'power': 0},
 143: {'power': 0},
 144: {'power': 0},
 145: {'power': 0},
 146: {'power': 0},
 147: {'power': 0},
 148: {'power': 0},
 149: {'power': 0},
 150: {'power': 0},
 151: {'power': 0},
 152: {'power': 0},
 153: {'power': 0},
 154: {'power': 0},
 155: {'power': 0},
 156: {'power': 0},
 157: {'power': 0},
 158: {'power': 0},
 159: {'power': 0},
 160: {'power': 0},
 161: {'power': 0},
 162: {'power': 0},
 163: {'power': 0},
 164: {'power': 0},
 165: {'power': 0},
 166: {'power': 0},
 167: {'power': 0},
 168: {'power': 0},
 169: {'power': 0},
 170: {'power': 0},
 171: {'power': 0},
 172: {'power': 0},
 173: {'power': 0},
 174: {'power': 0}}

In [121]:
for character_id, character in characters.items():
    for idx, item in enumerate(character['inventory']):
        item_id = item['item_id']
        if item_id in weapons:
            character['inventory'][idx]['power'] = weapons[item_id]['power']
        
        del character['inventory'][idx]['item_id']

In [123]:
characters[30]

{'character_id': 30,
 'name': 'Possimus ad dignissimos vel, a',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1,
 'fighter': {'using_shield': 0, 'rage': 100},
 'inventory': [{'name': 'Error eveniet ipsum, expedi',
   'value': 0,
   'weight': 0},
  {'name': 'Minus illum vo', 'value': 0, 'weight': 0},
  {'name': 'Sunt doloremque recusanda', 'value': 0, 'weight': 0},
  {'name': 'Corrupti sit at cum', 'value': 0, 'weight': 0, 'power': 0}]}

In [124]:
import json
with open('rpg.json', 'w') as fp:
    json.dump(characters, fp)